# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np


from bgcellmodels.common import analysis
from bgcellmodels.mechanisms import noise, synapses

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

In [ ]:
# Load cell model
from bgcellmodels.models.interneuron.Golomb2007_FSI import golomb_pynn_model
icell = h.GolombFSI()
soma = icell.soma[0]

# Fix ion channel density to Golomb (2007) value
soma.gkd_KdFSI = 0.00039

# Recording

In [ ]:
rec_secs = {
    'soma': soma,
}

trace_specs = {
    # Membrane voltages
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
    # Ion channels
    # 'i_Nas': {'var':'ina', 'mech': 'NasFSI', 'sec':'soma', 'loc':0.5},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Custom recordings
vrec = h.Vector()
vrec.record(soma(0.5)._ref_v)
trec = h.Vector()
trec.record(h._ref_t)

# Protocol: Spontaneous Activity

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038 # from Golomb article code
h.tstop = 1000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-80, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

# Protocol: Fig 2A

In [ ]:
stim = h.IClamp(soma(0.5))
stim.amp = 0.00335 # [nA] adjusted for area, corresponds to 3.35 uA/cm2
stim.delay = 48
stim.dur = 1e9

soma.gkd_KdFSI = 0.00039

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038 # from Golomb article code
h.tstop = 3000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-100, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Delayed tonic firing (Fig. 2A)')

# Protocol: Fig 2B

In [ ]:
soma.gkd_KdFSI = 0.0018
stim.amp = 0.0042

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -70.038 # from Golomb article code
h.tstop = 3000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-100, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Tonic stuttering (Fig. 2B)')